In [2]:
# Importing the required libraries

import pandas as pd
from bs4 import BeautifulSoup #Python library for pulling data out of HTML and XML files
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import os
import time

In [3]:
# Importing the required libraries

import numpy as np
import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests
import json
from pandas import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

In [51]:
# define the url to extract the data from

url_thane = 'https://www.magicbricks.com/localities-in-thane'
url_navimumbai = 'https://www.magicbricks.com/localities-in-Navi-Mumbai'

In [52]:
# Define the function to get the raw data form a url
# As each url uses dynamically loading page on scrolling down, we are using 'selenium' to get the compelete data

def magicbricksparser(url):
    options = webdriver.ChromeOptions() 
    options.add_argument("start-maximized")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(options=options, executable_path='C:\\Program Files\\Google\\Chrome\\Application\\chromedriver.exe')
    driver.get(url)
    WebDriverWait(driver, 20)
    #driver.quit()

    elem = driver.find_element_by_tag_name("body")

    no_of_pagedowns = 200

    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.2)
        no_of_pagedowns-=1
        
    return driver

In [53]:
# Run the funtions for thane and navi mumbai

magicbricks_thane = magicbricksparser(url_thane)
magicbricks_navimumbai = magicbricksparser(url_navimumbai)

In [54]:
#Using BeautifulSoup to parse the data.

soup_thane = BeautifulSoup(magicbricks_thane.page_source, "html.parser")

soup_navimumbai = BeautifulSoup(magicbricks_navimumbai.page_source, "html.parser")

In [55]:
print(soup_thane.prettify())
print(soup_navimumbai.prettify())

<html lang="en">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0, minimum-scale=1.0" name="viewport"/>
  <link href="//cdn.staticmb.com" rel="dns-prefetch preconnect"/>
  <link href="//img.staticmb.com" rel="dns-prefetch preconnect"/>
  <link as="font" crossorigin="crossorigin" href="https://cdn.staticmb.com/font/open-sans/Regular/OpenSans-Regular.woff2?v=1.1.0" rel="preload" type="font/woff2"/>
  <link as="font" crossorigin="crossorigin" href="https://cdn.staticmb.com/font/open-sans/Semibold/OpenSans-Semibold.woff2?v=1.1.0" rel="preload" type="font/woff2"/>
  <link as="font" crossorigin="crossorigin" href="https://cdn.staticmb.com/font/open-sans/Bold/OpenSans-Bold.woff2?v=1.1.0" rel="preload" type="font/woff2"/>
  <style type="text/css">
   @font-face {
	  font-family: 'Open Sans';
	  src: url("https://cdn.staticmb.com/font/open-sans/Regular/OpenSans-Regular.eot?v=

<html lang="en">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0, minimum-scale=1.0" name="viewport"/>
  <link href="//cdn.staticmb.com" rel="dns-prefetch preconnect"/>
  <link href="//img.staticmb.com" rel="dns-prefetch preconnect"/>
  <link as="font" crossorigin="crossorigin" href="https://cdn.staticmb.com/font/open-sans/Regular/OpenSans-Regular.woff2?v=1.1.0" rel="preload" type="font/woff2"/>
  <link as="font" crossorigin="crossorigin" href="https://cdn.staticmb.com/font/open-sans/Semibold/OpenSans-Semibold.woff2?v=1.1.0" rel="preload" type="font/woff2"/>
  <link as="font" crossorigin="crossorigin" href="https://cdn.staticmb.com/font/open-sans/Bold/OpenSans-Bold.woff2?v=1.1.0" rel="preload" type="font/woff2"/>
  <style type="text/css">
   @font-face {
	  font-family: 'Open Sans';
	  src: url("https://cdn.staticmb.com/font/open-sans/Regular/OpenSans-Regular.eot?v=

In [75]:
# create an empty dataframe
columns = ['Neighbourhood', 'ProjectStatus']
df = pd.DataFrame(columns=columns)
df

,Neighbourhood,ProjectStatus


In [57]:
# Define a function to a dictionary with a list of neighborhoods and if any residential areas are avilable in the neighborhoods

def createneighborhooddict(soup):
    
    neighbourhood_dict = {}

    for div_left_content in soup.find_all("div", attrs={"class":"left-content-wrapper"}):
        for div_loc_card in div_left_content.find_all("div", attrs={"class":"loc-card"}):

            div_card_title = div_loc_card.find("div", attrs={"class": "flex loc-card__title-n-map"})

            title = div_card_title.find("a", attrs={"class":"loc-card__title"})
            neighbourhood_dict[title.text] = {}
            #print(title.text)

            div_no_project_wrap = div_loc_card.find("div", attrs={"class": "noprojectavailWrap"})

            if div_no_project_wrap is None:
                neighbourhood_dict[title.text]["project"] = "Projects Available"
            else:
                div_no_project = div_no_project_wrap.find("div", attrs={"class": "noproject"})
                neighbourhood_dict[title.text]["project"] = div_no_project.text
    
    
    return neighbourhood_dict

In [62]:
# Run the function for Thane and Navi Mumbai
# Merge the data into one dictionary

neighbourhood_dict = {}

neighbourhood_dict_thane = createneighborhooddict(soup_thane)

neighbourhood_dict_navimumbai = createneighborhooddict(soup_navimumbai)

neighbourhood_dict.update(neighbourhood_dict_thane)
neighbourhood_dict.update(neighbourhood_dict_navimumbai)

In [64]:
# Final dictionary created after merge

neighbourhood_dict

{'Thane West, Thane': {'project': 'Projects Available'},
 'Ghodbunder Road, Thane': {'project': 'Projects Available'},
 'Dombivli, Thane': {'project': 'Projects Available'},
 'Majiwada, Thane': {'project': 'Projects Available'},
 'Kalyan, Thane': {'project': 'Projects Available'},
 'Dombivli East, Thane': {'project': 'Projects Available'},
 'Hiranandani Estate, Thane': {'project': 'Projects Available'},
 'Kolshet Road, Thane': {'project': 'Projects Available'},
 'Kalyan West, Thane': {'project': 'Projects Available'},
 'Manpada, Thane': {'project': 'Projects Available'},
 'Badlapur, Thane': {'project': 'Projects Available'},
 'Kasarvadavali, Thane': {'project': 'Projects Available'},
 'Ambernath, Thane': {'project': 'Projects Available'},
 'Balkum village, Thane': {'project': 'Projects Available'},
 'Naupada, Thane': {'project': 'Projects Available'},
 'Pokharan Road No. 2, Thane': {'project': 'Projects Available'},
 'Palava, Thane': {'project': 'Projects Available'},
 'Badlapur East, 

In [76]:
# populate dataframe with data from dictionary

for ind, neighbourhood in enumerate(neighbourhood_dict):
    project_status = neighbourhood_dict[neighbourhood]['project']
    
    df = df.append({"Neighbourhood": neighbourhood, "ProjectStatus": project_status}, ignore_index='True')
    
df

,Neighbourhood,ProjectStatus
0,"Thane West, Thane",Projects Available
1,"Ghodbunder Road, Thane",Projects Available
2,"Dombivli, Thane",Projects Available
3,"Majiwada, Thane",Projects Available
4,"Kalyan, Thane",Projects Available
...,...,...
427,"Dattatray Tandel Marg, Navi Mumbai",No Projects available in this locality
428,"Sector 16A, Navi Mumbai",No Projects available in this locality
429,"Sector 48 A, Navi Mumbai",No Projects available in this locality
430,"Vadghar, Navi Mumbai",No Projects available in this locality


In [78]:
# Selecting the neighborhoods where residential areas are avialable

df = df[df['ProjectStatus'] == 'Projects Available'].reset_index(drop=True)

In [129]:
df

,Neighbourhood,ProjectStatus
0,"Thane West, Thane",Projects Available
1,"Ghodbunder Road, Thane",Projects Available
3,"Majiwada, Thane",Projects Available
4,"Kalyan, Thane",Projects Available
6,"Hiranandani Estate, Thane",Projects Available
...,...,...
90,"Rabale, Navi Mumbai",Projects Available
91,"Navade, Navi Mumbai",Projects Available
92,"Ulwe Village, Navi Mumbai",Projects Available
93,"Sector 36, Navi Mumbai",Projects Available


In [130]:
# Remove areas from the list which are too far out from the center
# Removing all areas with specific words in the Neighorhood list


remove_areas_list = ['Vasind', 'Atgaon', 'Asangaon', 'Titwala', 'Kalyan East', 'Ulhas', 'Ambivli', 'Shahad', 'Ambernath', 'Badlapur', 'Karjat', 'Neral', 'Dombivli', 'Thakurli', 'Gokhale', 'Dronagiri','Rasayani','Navade','Mosare']

for value in remove_areas_list:
    
    index_names = df[df["Neighbourhood"].str.contains(value)].index
    df.drop(index_names, inplace = True)
#index_names

df

,Neighbourhood,ProjectStatus
0,"Thane West, Thane",Projects Available
1,"Ghodbunder Road, Thane",Projects Available
3,"Majiwada, Thane",Projects Available
4,"Kalyan, Thane",Projects Available
6,"Hiranandani Estate, Thane",Projects Available
...,...,...
88,"Sector 12-Khargar, Navi Mumbai",Projects Available
89,"Khandeshwar, Navi Mumbai",Projects Available
90,"Rabale, Navi Mumbai",Projects Available
92,"Ulwe Village, Navi Mumbai",Projects Available


In [131]:
df.reset_index(drop=True)

,Neighbourhood,ProjectStatus
0,"Thane West, Thane",Projects Available
1,"Ghodbunder Road, Thane",Projects Available
2,"Majiwada, Thane",Projects Available
3,"Kalyan, Thane",Projects Available
4,"Hiranandani Estate, Thane",Projects Available
...,...,...
65,"Sector 12-Khargar, Navi Mumbai",Projects Available
66,"Khandeshwar, Navi Mumbai",Projects Available
67,"Rabale, Navi Mumbai",Projects Available
68,"Ulwe Village, Navi Mumbai",Projects Available


In [132]:
# export dataframe to file for future reference

df.to_csv('thane_res_neighbourhoods.txt')

In [133]:
res_sector_list_thane = pd.read_csv('thane_res_neighbourhoods.txt')
res_sector_list_thane.drop(columns='Unnamed: 0', axis=1, inplace=True)
res_sector_list_thane

,Neighbourhood,ProjectStatus
0,"Thane West, Thane",Projects Available
1,"Ghodbunder Road, Thane",Projects Available
2,"Majiwada, Thane",Projects Available
3,"Kalyan, Thane",Projects Available
4,"Hiranandani Estate, Thane",Projects Available
...,...,...
65,"Sector 12-Khargar, Navi Mumbai",Projects Available
66,"Khandeshwar, Navi Mumbai",Projects Available
67,"Rabale, Navi Mumbai",Projects Available
68,"Ulwe Village, Navi Mumbai",Projects Available


In [134]:
# run a for loop to get lantitude and longitude for all sectors and store in data in a dictionary

location_dict = {}


for value in res_sector_list_thane['Neighbourhood']:
    address = value
    location_dict[value] = {}
    

    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    
    if location is None:
        latitude = None
        longitude = None
    
    else:
        latitude = location.latitude
        longitude = location.longitude
    
    location_dict[value]['Latitude'] = latitude
    location_dict[value]['Longitude'] = longitude

In [135]:
location_dict

{'Thane West, Thane': {'Latitude': 19.1943294, 'Longitude': 72.9701779},
 'Ghodbunder Road, Thane': {'Latitude': 19.2882132, 'Longitude': 72.8955745},
 'Majiwada, Thane': {'Latitude': 19.2130251, 'Longitude': 72.9784852},
 'Kalyan, Thane': {'Latitude': 19.2716301, 'Longitude': 73.23586552517415},
 'Hiranandani Estate, Thane': {'Latitude': 19.256855950000002,
  'Longitude': 72.98427051574063},
 'Kolshet Road, Thane': {'Latitude': 19.24332, 'Longitude': 72.9983196},
 'Kalyan West, Thane': {'Latitude': 19.2716301,
  'Longitude': 73.23586552517415},
 'Manpada, Thane': {'Latitude': 19.2314156, 'Longitude': 72.9738156},
 'Kasarvadavali, Thane': {'Latitude': 19.1452859, 'Longitude': 72.9352214},
 'Balkum village, Thane': {'Latitude': 19.22169075,
  'Longitude': 72.9844924192587},
 'Naupada, Thane': {'Latitude': 19.1890469, 'Longitude': 72.9692339},
 'Pokharan Road No. 2, Thane': {'Latitude': None, 'Longitude': None},
 'Palava, Thane': {'Latitude': 19.164609149999997,
  'Longitude': 73.1127841

In [136]:
# checking if location data is not avilable for any neighborhood in Nominatum database
# Generate a list of such neighborhoods

null_address_list = []

for ind, neighbourhood in enumerate(location_dict):
    if location_dict[neighbourhood]['Latitude'] is None or location_dict[neighbourhood]['Longitude'] is None:
        null_address_list.append(neighbourhood)

null_address_list

['Pokharan Road No. 2, Thane',
 'Bhayanderpada, Thane',
 'Owale, Thane',
 'Kasheli, Thane',
 'Pokharan Rd No 1, Thane',
 'Taloja, Navi Mumbai',
 'Karanjade, Navi Mumbai',
 'Road Pali, Navi Mumbai',
 'Palm Beach Road, Navi Mumbai',
 'Taloje Panchanand, Navi Mumbai',
 'Khopoli, Navi Mumbai',
 'Sector 12-Khargar, Navi Mumbai']

In [138]:
# Search and location on google maps and update the dictionary
# Not using google api to get the location

latitude_list = [19.21925, 19.28787, 19.27355, 19.23748, 19.21345,19.06819,18.97342,19.03846,19.04562, 19.07112, 19.07265, 19.04256]
longitude_list = [72.96776, 72.95017, 72.95863, 73.01379, 72.95952,73.10428,73.10990,73.09485,73.01008, 73.09842, 73.02119, 73.06451]


In [139]:
# populating the data in the location dictionary

for value in null_address_list:
    for x,y in zip(latitude_list, longitude_list):
        location_dict[value]['Latitude'] = x
        location_dict[value]['Longitude'] = y

In [140]:
# create an empty dataframe

columns = ['Neighbourhood', 'Latitude', 'Longitude']
df_neighbourhood = pd.DataFrame(columns=columns)
df_neighbourhood

# populate dataframe with data from dictionary

for ind, neighbourhood in enumerate(location_dict):
    lat = location_dict[neighbourhood]['Latitude']
    long = location_dict[neighbourhood]['Longitude']
    
    df_neighbourhood = df_neighbourhood.append({"Neighbourhood": neighbourhood, "Latitude": lat, "Longitude": long}, ignore_index='True')
    
df_neighbourhood

,Neighbourhood,Latitude,Longitude
0,"Thane West, Thane",19.194329,72.970178
1,"Ghodbunder Road, Thane",19.288213,72.895574
2,"Majiwada, Thane",19.213025,72.978485
3,"Kalyan, Thane",19.271630,73.235866
4,"Hiranandani Estate, Thane",19.256856,72.984271
...,...,...,...
65,"Sector 12-Khargar, Navi Mumbai",19.042560,73.064510
66,"Khandeshwar, Navi Mumbai",19.007350,73.095283
67,"Rabale, Navi Mumbai",19.136636,73.002782
68,"Ulwe Village, Navi Mumbai",18.976861,73.020131


In [141]:
# export dataframe to file for future reference

df_neighbourhood.to_csv("thane_res_sectors_geolocator_data.txt")

In [142]:
# Setting the credential for accessing Foursquare API
# The credentials were removed are use for security purposes

CLIENT_ID =  # your Foursquare ID
CLIENT_SECRET =  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [143]:
#Create a function to explore each residential sector in Thane

def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = f"https://api.foursquare.com/v2/venues/explore?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&ll={lat},{lng}&v={VERSION}&radius={radius}&limit={LIMIT}"
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
    

In [144]:
# Runing the function. The data will be store in a pandas dataframe

thane_venues = getNearbyVenues(names=df_neighbourhood['Neighbourhood'], latitudes=df_neighbourhood['Latitude'], longitudes=df_neighbourhood['Longitude'])

Thane West, Thane
Ghodbunder Road, Thane
Majiwada, Thane
Kalyan, Thane
Hiranandani Estate, Thane
Kolshet Road, Thane
Kalyan West, Thane
Manpada, Thane
Kasarvadavali, Thane
Balkum village, Thane
Naupada, Thane
Pokharan Road No. 2, Thane
Palava, Thane
Anand Nagar, Thane
Waghbil, Thane
Kalwa, Thane
Teen Hath Naka, Thane
Dhokali, Thane
Vartak Nagar, Thane
Vasant Vihar, Thane
Pokhran Road, Thane
Shilphata, Thane
Bhiwandi, Thane
Kapurbawdi, Thane
Patlipada, Thane
Bhayanderpada, Thane
Hiranandani Meadows, Thane
Owale, Thane
Diva, Thane
Brahmand, Thane
Kasheli, Thane
Kolshet, Thane
Mumbra, Thane
Khopat, Thane
Shahapur, Thane
Palghar, Thane
Pokharan Rd No 1, Thane
Boisar, Thane
Jeejamata Nagar, Thane
Louis Wadi, Thane
Hiranandani Road, Thane
Murbad, Thane
Kausa, Thane
Panvel, Navi Mumbai
Kharghar, Navi Mumbai
Ulwe, Navi Mumbai
New Panvel, Navi Mumbai
Nerul, Navi Mumbai
Taloja, Navi Mumbai
Airoli, Navi Mumbai
Seawoods, Navi Mumbai
Kamothe, Navi Mumbai
Kopar Khairane, Navi Mumbai
Vashi, Navi Mumb

In [145]:
thane_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Thane West, Thane",19.194329,72.970178,Café Verve,19.192178,72.973612,Café
1,"Thane West, Thane",19.194329,72.970178,Mad Over Donuts,19.193368,72.967655,Donut Shop
2,"Thane West, Thane",19.194329,72.970178,Utsav Restaurant,19.194688,72.966683,Indian Restaurant
3,"Thane West, Thane",19.194329,72.970178,Stone Oven,19.192209,72.973502,Italian Restaurant
4,"Thane West, Thane",19.194329,72.970178,Preeti Sandwich,19.195162,72.976673,Sandwich Place


In [146]:
# export dataframe to file for future reference

thane_venues.to_csv('thane_res_sectors_foresquare_data_2000m.txt')

In [4]:
df_neighbourhood = pd.read_csv("thane_res_sectors_geolocator_data.txt")
df_neighbourhood.drop(columns='Unnamed: 0', axis=1, inplace=True)
df_neighbourhood.head()

,Neighbourhood,Latitude,Longitude
0,"Thane West, Thane",19.194329,72.970178
1,"Ghodbunder Road, Thane",19.288213,72.895574
2,"Majiwada, Thane",19.213025,72.978485
3,"Kalyan, Thane",19.271630,73.235866
4,"Hiranandani Estate, Thane",19.256856,72.984271


In [5]:
thane_venues = pd.read_csv('thane_res_sectors_foresquare_data_2000m.txt')
thane_venues.drop(columns='Unnamed: 0', axis=1, inplace=True)
thane_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Thane West, Thane",19.194329,72.970178,Café Verve,19.192178,72.973612,Café
1,"Thane West, Thane",19.194329,72.970178,Mad Over Donuts,19.193368,72.967655,Donut Shop
2,"Thane West, Thane",19.194329,72.970178,Utsav Restaurant,19.194688,72.966683,Indian Restaurant
3,"Thane West, Thane",19.194329,72.970178,Stone Oven,19.192209,72.973502,Italian Restaurant
4,"Thane West, Thane",19.194329,72.970178,Preeti Sandwich,19.195162,72.976673,Sandwich Place


In [149]:
# Check number of venues returned for each neighbourhood

thane_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Airoli, Navi Mumbai",24,24,24,24,24,24
"Anand Nagar, Thane",86,86,86,86,86,86
"Balkum village, Thane",64,64,64,64,64,64
"Belapur, Navi Mumbai",58,58,58,58,58,58
"Bhayanderpada, Thane",43,43,43,43,43,43
...,...,...,...,...,...,...
"Uran, Navi Mumbai",44,44,44,44,44,44
"Vartak Nagar, Thane",87,87,87,87,87,87
"Vasant Vihar, Thane",80,80,80,80,80,80


In [11]:
# Number of unique venue categories

len(thane_venues['Venue Category'].unique())

135

In [12]:
# one hot encoding
thane_onehot = pd.get_dummies(thane_venues['Venue Category'])

# add neighborhood column to the dataframe
thane_onehot['Neighborhood'] = thane_venues['Neighborhood']

# move neighborhood column to the first column
first_column = thane_onehot.pop('Neighborhood')
thane_onehot.insert(0, 'Neighborhood', first_column)

thane_onehot.head()

,Neighborhood,ATM,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,...,Toll Plaza,Toy / Game Store,Track Stadium,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Water Park,Wine Bar,Wine Shop
0,"Thane West, Thane",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Thane West, Thane",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Thane West, Thane",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Thane West, Thane",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Thane West, Thane",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Group the data by neighbourhood and use mean as the required function.

thane_grouped = thane_onehot.groupby('Neighborhood').mean().reset_index()
thane_grouped

,Neighborhood,ATM,American Restaurant,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,...,Toll Plaza,Toy / Game Store,Track Stadium,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Water Park,Wine Bar,Wine Shop
0,"Airoli, Navi Mumbai",0.0,0.000000,0.000000,0.0,0.041667,0.000000,0.000000,0.000000,0.041667,...,0.0,0.041667,0.000000,0.0,0.000000,0.041667,0.000000,0.0000,0.0,0.0
1,"Anand Nagar, Thane",0.0,0.000000,0.000000,0.0,0.011628,0.011628,0.000000,0.000000,0.023256,...,0.0,0.000000,0.000000,0.0,0.011628,0.023256,0.000000,0.0000,0.0,0.0
2,"Balkum village, Thane",0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.046875,...,0.0,0.000000,0.000000,0.0,0.000000,0.015625,0.000000,0.0000,0.0,0.0
3,"Belapur, Navi Mumbai",0.0,0.000000,0.000000,0.0,0.034483,0.000000,0.017241,0.000000,0.034483,...,0.0,0.000000,0.000000,0.0,0.034483,0.017241,0.000000,0.0000,0.0,0.0
4,"Bhayanderpada, Thane",0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.023256,...,0.0,0.000000,0.000000,0.0,0.023256,0.000000,0.000000,0.0000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,"Uran, Navi Mumbai",0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.045455,...,0.0,0.000000,0.000000,0.0,0.022727,0.022727,0.000000,0.0000,0.0,0.0
64,"Vartak Nagar, Thane",0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.011494,0.011494,0.011494,...,0.0,0.011494,0.000000,0.0,0.000000,0.011494,0.011494,0.0000,0.0,0.0
65,"Vasant Vihar, Thane",0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.025000,...,0.0,0.012500,0.000000,0.0,0.000000,0.012500,0.000000,0.0125,0.0,0.0
66,"Vashi, Navi Mumbai",0.0,0.010204,0.010204,0.0,0.010204,0.000000,0.000000,0.000000,0.020408,...,0.0,0.000000,0.010204,0.0,0.000000,0.010204,0.000000,0.0000,0.0,0.0


In [14]:
k = 6 # Defining cluster size

thane_grouped_clustering = thane_grouped.drop('Neighborhood', axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0)
kmeans.fit(thane_grouped_clustering)

KMeans(n_clusters=6, random_state=0)

In [154]:
# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 3, 3, 0, 0, 0, 3, 3, 0, 3, 0, 0, 0, 3, 0, 4, 4, 3, 0,
       3, 3, 3, 3, 3, 3, 0, 3, 5, 0, 0, 0, 0, 0, 3, 2, 0, 0, 0, 3, 1, 0,
       3, 0, 0, 3, 3, 0, 3, 3, 0, 0, 3, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [15]:
# check number of values for each cluster

np.unique(kmeans.labels_, return_counts=True)

(array([0, 1, 2, 3, 4, 5]), array([39,  1,  1, 24,  2,  1], dtype=int64))

In [16]:
# Define function to get top venues for each neighborhood

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 10 # Define number of top venues to be extracted

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = thane_grouped['Neighborhood']

for ind in np.arange(thane_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(thane_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Airoli, Navi Mumbai",Fast Food Restaurant,Pizza Place,Indian Restaurant,Gym,Department Store,Hotel Bar,Restaurant,Cocktail Bar,Seafood Restaurant,Chinese Restaurant
1,"Anand Nagar, Thane",Indian Restaurant,Fast Food Restaurant,Ice Cream Shop,Dessert Shop,Pizza Place,Chinese Restaurant,Snack Place,Italian Restaurant,Café,Bakery
2,"Balkum village, Thane",Pizza Place,Indian Restaurant,Coffee Shop,Fast Food Restaurant,Shopping Mall,Multiplex,Bakery,Department Store,Clothing Store,Chinese Restaurant
3,"Belapur, Navi Mumbai",Indian Restaurant,Café,Fast Food Restaurant,Pizza Place,Dessert Shop,Chinese Restaurant,Restaurant,Train Station,Coffee Shop,Asian Restaurant
4,"Bhayanderpada, Thane",Café,Indian Restaurant,Pizza Place,Fast Food Restaurant,Multiplex,Restaurant,Gym,Department Store,Convenience Store,Sandwich Place


In [18]:
neighborhoods_venues_sorted.shape

(68, 11)

In [20]:
# Insert cluster labels at position '0' to the dataframe

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Airoli, Navi Mumbai",Fast Food Restaurant,Pizza Place,Indian Restaurant,Gym,Department Store,Hotel Bar,Restaurant,Cocktail Bar,Seafood Restaurant,Chinese Restaurant
1,0,"Anand Nagar, Thane",Indian Restaurant,Fast Food Restaurant,Ice Cream Shop,Dessert Shop,Pizza Place,Chinese Restaurant,Snack Place,Italian Restaurant,Café,Bakery
2,0,"Balkum village, Thane",Pizza Place,Indian Restaurant,Coffee Shop,Fast Food Restaurant,Shopping Mall,Multiplex,Bakery,Department Store,Clothing Store,Chinese Restaurant
3,0,"Belapur, Navi Mumbai",Indian Restaurant,Café,Fast Food Restaurant,Pizza Place,Dessert Shop,Chinese Restaurant,Restaurant,Train Station,Coffee Shop,Asian Restaurant
4,3,"Bhayanderpada, Thane",Café,Indian Restaurant,Pizza Place,Fast Food Restaurant,Multiplex,Restaurant,Gym,Department Store,Convenience Store,Sandwich Place


In [31]:
thane_merged = df_neighbourhood
thane_merged.rename(columns = {'Neighbourhood': 'Neighborhood'}, inplace=True)
thane_merged.head()

,Neighborhood,Latitude,Longitude
0,"Thane West, Thane",19.194329,72.970178
1,"Ghodbunder Road, Thane",19.288213,72.895574
2,"Majiwada, Thane",19.213025,72.978485
3,"Kalyan, Thane",19.271630,73.235866
4,"Hiranandani Estate, Thane",19.256856,72.984271


In [32]:
# Join the dataframe generated after getting data from geolocator with the data generated after clustering

thane_merged = thane_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
thane_merged

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Thane West, Thane",19.194329,72.970178,0.0,Indian Restaurant,Fast Food Restaurant,Ice Cream Shop,Coffee Shop,Seafood Restaurant,Chinese Restaurant,Hotel,Clothing Store,Dessert Shop,Italian Restaurant
1,"Ghodbunder Road, Thane",19.288213,72.895574,0.0,Indian Restaurant,Restaurant,Food Court,Smoke Shop,Diner,Gym / Fitness Center,Food,Resort,Dhaba,Hookah Bar
2,"Majiwada, Thane",19.213025,72.978485,0.0,Indian Restaurant,Coffee Shop,Fast Food Restaurant,Pizza Place,Shopping Mall,Seafood Restaurant,Multiplex,Clothing Store,Sandwich Place,Bakery
3,"Kalyan, Thane",19.271630,73.235866,4.0,Hotel,Trail,Wine Shop,Electronics Store,Food & Drink Shop,Food,Flea Market,Fast Food Restaurant,Farmers Market,English Restaurant
4,"Hiranandani Estate, Thane",19.256856,72.984271,3.0,Bus Station,Grocery Store,Café,Department Store,Ice Cream Shop,Restaurant,Coffee Shop,Food,Shopping Mall,Chinese Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,"Sector 12-Khargar, Navi Mumbai",19.042560,73.064510,3.0,Café,Indian Restaurant,Pizza Place,Fast Food Restaurant,Multiplex,Restaurant,Gym,Department Store,Convenience Store,Sandwich Place
66,"Khandeshwar, Navi Mumbai",19.007350,73.095283,3.0,Asian Restaurant,Department Store,Dog Run,Bus Station,Movie Theater,Fast Food Restaurant,Multiplex,Toll Plaza,Gym,Ice Cream Shop
67,"Rabale, Navi Mumbai",19.136636,73.002782,3.0,Department Store,Fast Food Restaurant,Café,Pizza Place,Gym,Platform,Dumpling Restaurant,Coffee Shop,Sandwich Place,Seafood Restaurant
68,"Ulwe Village, Navi Mumbai",18.976861,73.020131,0.0,Paper / Office Supplies Store,Playground,Home Service,Athletics & Sports,Historic Site,Mobile Phone Shop,Wine Shop,Flea Market,Fast Food Restaurant,Farmers Market


In [33]:
# check if any data from gelocator data had no response from Fourequare and thus, was not used in clustering

thane_merged1 = thane_merged[thane_merged.isnull().any(axis=1)]
thane_merged1

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,"Bhiwandi, Thane",19.362531,73.078475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,"Shahapur, Thane",19.516252,73.419003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [169]:
thane_merged.shape

(70, 14)

In [170]:
# Select only the data which was used in clustering and we received data from Foursquare

thane_merged = thane_merged[thane_merged['Cluster Labels'].isnull() == False]
thane_merged.shape

(68, 14)

In [171]:
address = 'Thane, Maharashtra'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [172]:
# create map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(thane_merged['Latitude'], thane_merged['Longitude'], thane_merged['Neighborhood'], thane_merged['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters